In [3]:
import psycopg2
import pandas as pd

In [4]:
conn = psycopg2.connect(
            host='localhost',
            port='5432',
            database='airflow',
            user='postgres',
            password='MYZv3Ietuflo'
        )
cur = conn.cursor()

In [ ]:
conn.close()

In [5]:
cur.execute(f'select * from departures')
result = cur.fetchall()
result

[('AA',
  311,
  'DCA',
  datetime.time(20, 59),
  datetime.date(2021, 11, 1),
  datetime.datetime(2021, 11, 1, 20, 0),
  0),
 ('AA',
  328,
  'DFW',
  datetime.time(5, 15),
  datetime.date(2021, 11, 1),
  datetime.datetime(2021, 11, 1, 5, 0),
  -4),
 ('AA',
  329,
  'IAH',
  datetime.time(19, 35),
  datetime.date(2021, 11, 1),
  datetime.datetime(2021, 11, 1, 19, 0),
  31),
 ('AA',
  357,
  'LGA',
  datetime.time(20, 30),
  datetime.date(2021, 11, 1),
  datetime.datetime(2021, 11, 1, 20, 0),
  0),
 ('AA',
  359,
  'LGA',
  datetime.time(17, 0),
  datetime.date(2021, 11, 1),
  datetime.datetime(2021, 11, 1, 17, 0),
  -10),
 ('AA',
  361,
  'PHX',
  datetime.time(6, 0),
  datetime.date(2021, 11, 1),
  datetime.datetime(2021, 11, 1, 6, 0),
  0),
 ('AA',
  374,
  'LAX',
  datetime.time(19, 46),
  datetime.date(2021, 11, 1),
  datetime.datetime(2021, 11, 1, 19, 0),
  33),
 ('AA',
  378,
  'PHL',
  datetime.time(13, 23),
  datetime.date(2021, 11, 1),
  datetime.datetime(2021, 11, 1, 13, 0),

In [72]:
cur.execute(
    f'''
    create table if not exists departures (
        carrier_code text,
        flight int,
        destination_airport text,
        scheduled_datetime time,
        date date,
        datetime_round timestamp,
        delay int);
    '''
)

cur.execute(f'select * from departures')
result = cur.fetchall()
result

[]

In [73]:
cur.execute(
        f'''insert into departures (carrier_code, flight, destination_airport, 
            scheduled_datetime, date, datetime_round, delay) 
            values (\'VX\', 201, \'SFO\', \'07:00:00\', \'2012-01-01\', TIMESTAMP \'2012-01-01 07:00:00\', 3);'''
        )
conn.commit()

In [6]:
with open(
    '/home/ivan_admin/dmls_aviaticket/ipynb/online_parsing/flights_2023-01-16.csv'
    ) as file:
    csv = file.readlines()

In [7]:
csv[:5]

[',Date,Flight,Carrier Code,Scheduled Time,Delay,Destination airport\n',
 '0,2023-01-16,Y151H,AX,00:00,62,CDG\n',
 '1,2023-01-16,7458,Y8,00:01,,ANC\n',
 '2,2023-01-16,7456,Y8,00:01,,ANC\n',
 '3,2023-01-16,7452,Y8,00:01,,ANC\n']

In [19]:
''.join([n for n in 'sdf12sgsd' if n.isdigit()])

'12'

In [8]:
table_data = []
for data in csv[1:]:
    data = data.replace('\n', '').split(',')[1:]
    #data[3] = data[3].split(':')[0]
    data[1] = ''.join([n for n in data[1] if n.isdigit()])
    #data[0] = '2023-01-12'
    data[4] = 'NULL' if data[4] == '' else data[4]
    string = f'''\'{data[2]}\', {data[1]}, \'{data[5]}\', \'{data[3]}:00\', \'{data[0]}\', TIMESTAMP \'{data[0] + ' ' + data[3].split(':')[0]}:00:00\', {data[4]}'''
    table_data.append(string)

columns = 'carrier_code, flight, destination_airport, scheduled_datetime, date, datetime_round, delay'
query = f"""insert into departures ({columns}) \nvalues """
for row in table_data:
    query += f'\n({row}), '
query = query[:-2] + ';'

In [9]:
print(query[:500])

insert into departures (carrier_code, flight, destination_airport, scheduled_datetime, date, datetime_round, delay) 
values 
('AX', 151, 'CDG', '00:00:00', '2023-01-16', TIMESTAMP '2023-01-16 00:00:00', 62), 
('Y8', 7458, 'ANC', '00:01:00', '2023-01-16', TIMESTAMP '2023-01-16 00:00:00', NULL), 
('Y8', 7456, 'ANC', '00:01:00', '2023-01-16', TIMESTAMP '2023-01-16 00:00:00', NULL), 
('Y8', 7452, 'ANC', '00:01:00', '2023-01-16', TIMESTAMP '2023-01-16 00:00:00', NULL), 
('Y8', 7454, 'ANC', '00:01:00'


In [10]:
cur.execute(query)
conn.commit()

In [14]:
cur.execute('select count(*) from departures')
result = cur.fetchall()
result

[(264272,)]

## Теперь запишем новую таблицу с тестом

In [43]:
cur.execute(
    f'''
    create table if not exists departures_to_test (
        carrier_code text,
        flight int,
        destination_airport text,
        scheduled_datetime time,
        date date,
        datetime_round timestamp,
        delay int);
    '''
)

cur.execute(f'select * from departures_to_test')
result = cur.fetchall()
result

[]

In [11]:
with open(
    '/home/ivan_admin/dmls_aviaticket/ipynb/online_parsing/flights_2023-01-18.csv'
    ) as file:
    csv = file.readlines()

In [12]:
table_data = []
for data in csv[1:]:
    data = data.replace('\n', '').split(',')[1:]
    #data[3] = data[3].split(':')[0]
    data[1] = ''.join([n for n in data[1] if n.isdigit()])
    #data[0] = '2023-01-12'
    data[4] = 'NULL'
    data[4] = 0 if data[4] == '' else data[4]
    string = f'''\'{data[2]}\', {data[1]}, \'{data[5]}\', \'{data[3]}:00\', \'{data[0]}\', TIMESTAMP \'{data[0] + ' ' + data[3].split(':')[0]}:00:00\', {data[4]}'''
    table_data.append(string)

columns = 'carrier_code, flight, destination_airport, scheduled_datetime, date, datetime_round, delay'
query = f"""insert into departures_to_test ({columns}) \nvalues """
for row in table_data:
    query += f'\n({row}), '
query = query[:-2] + ';'

In [13]:
print(query[:500])

insert into departures_to_test (carrier_code, flight, destination_airport, scheduled_datetime, date, datetime_round, delay) 
values 
('Y8', 7456, 'ANC', '00:01:00', '2023-01-18', TIMESTAMP '2023-01-18 00:00:00', NULL), 
('Y8', 7452, 'ANC', '00:01:00', '2023-01-18', TIMESTAMP '2023-01-18 00:00:00', NULL), 
('Y8', 7454, 'ANC', '00:01:00', '2023-01-18', TIMESTAMP '2023-01-18 00:00:00', NULL), 
('Y8', 7458, 'ANC', '00:01:00', '2023-01-18', TIMESTAMP '2023-01-18 00:00:00', NULL), 
('VB', 547, 'GDL', 


In [14]:
cur.execute(query)
conn.commit()